In [1]:
import os
import pandas as pd
import seaborn as sns
from scipy.stats import norm
import math as mt
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom
import warnings
import itertools
import datetime
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
import pandas as pd
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston

In [2]:
sns.set()
import warnings
warnings.filterwarnings('ignore')
path_store='C:/Users/chuna/Downloads/week5/store.csv'
path_train='C:/Users/chuna/Downloads/week5/train.csv'
path_test='C:/Users/chuna/Downloads/week5/test.csv'
isFile = os.path.isfile(path_train) 
print(isFile)                           

df_store = pd.read_csv(path_store)
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)



df_train=df_train.sample(frac =.9)
df_train = df_train.drop('Customers', 1)
df_train

True


,Store,DayOfWeek,Date,Sales,Open,Promo,StateHoliday,SchoolHoliday
656940,991,3,2013-11-20,5430,1,1,0,0
264573,178,1,2014-12-01,17449,1,1,0,0
406764,261,3,2014-07-02,15924,1,1,0,0
370352,325,7,2014-08-10,0,0,0,0,0
162618,944,7,2015-03-08,0,0,0,0,0
...,...,...,...,...,...,...,...,...
273687,1006,6,2014-11-22,6847,1,0,0,0
884171,762,2,2013-04-30,4970,1,1,0,0
472921,947,7,2014-05-04,0,0,0,0,0
101262,913,6,2015-05-02,10847,1,0,0,0


In [3]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [4]:
df_train["Date"]= pd.to_datetime(df_train["Date"]) 
df_train["Date"]=df_train["Date"].astype('datetime64[D]')
holydates=df_train.loc[((df_train['StateHoliday'] != '0'))]['Date'].unique()


In [5]:
def nearest(pivot):
    return min(holydates, key=lambda x: abs(x - pivot))

In [6]:
def convert2int(df_train):

    df_train['Store'] = df_train['Store'].astype(int)
    df_train['DayOfWeek'] = df_train['DayOfWeek'].astype(int)
    #df_train['Customers'] = df_train['Customers'].astype(int)
    
    
    df_train['Open'] = df_train['Open'].astype(int)
    df_train['Promo'] = df_train['Promo'].astype(int)
    df_train['SchoolHoliday'] = df_train['SchoolHoliday'].astype(int)
    return df_train

In [7]:
def convert2date(df_train):

    df_train["Date"]= pd.to_datetime(df_train["Date"]) 
    df_train["Date"]=df_train["Date"].astype('datetime64[D]')
    holydates=df_train.loc[((df_train['StateHoliday'] != '0'))]['Date'].unique()
    return df_train

In [8]:
def new_insights(df_train):

    df_train['day'] = df_train['Date'].dt.day_name() 
    df_train['month'] = df_train['Date'].dt.month_name() 
    df_train['year'] = df_train['Date'].dt.year  
    df_train.loc[((df_train['day'] == 'Saturday') |(df_train['day'] == 'Sunday')  ) , 'weekday'] = 'weekend' 
    df_train['weekday']=df_train['weekday'].fillna('weekday')
    df_train['StateHoliday'] = df_train['StateHoliday'].astype(str)
    


    df_train['Difference_to_StateHoliday'] = df_train['Date'].apply(nearest)

    df_train['Difference_to_StateHoliday'] = df_train['Date'].sub(df_train['Difference_to_StateHoliday'], axis=0)

    return df_train

In [9]:
def new_insights2(df_train):

    resort=df_train.copy()
    resort['Difference_to_StateHoliday'] = resort['Difference_to_StateHoliday'].astype(str)

    resort["Difference_to_StateHoliday"]= resort["Difference_to_StateHoliday"].str.split("d", 1)
    resort['Difference_to_StateHoliday'] = resort["Difference_to_StateHoliday"].str.get(0) 
    resort.sample(3)
    resort['Difference_to_StateHoliday'] = resort['Difference_to_StateHoliday'].astype(str)

    resort['Difference_to_StateHoliday'] =resort.Difference_to_StateHoliday.mask(resort.Difference_to_StateHoliday.str[-1].isin(['-','+']), resort.Difference_to_StateHoliday.str[-1].str.cat(resort.Difference_to_StateHoliday.str[:-1])).astype('float')


    resort['Difference_to_StateHoliday'] = resort['Difference_to_StateHoliday'].astype(int)
    df_train=resort
    return df_train

In [10]:

# read the training data set
data = df_train

# top rows of the data
data.head()

# seperate the independent and target variables
train_x = data.drop(columns=['Sales'])
train_y = data['Sales']

In [11]:
# import the BaseEstimator
from sklearn.base import BaseEstimator

# define the class OutletTypeEncoder
# This will be our custom transformer that will create 3 new binary columns
# custom transformer must have methods fit and transform
class OutletTypeEncoder(BaseEstimator):

    def __init__(self):
        pass

    def fit(self, documents, y=None):
        return self

    def transform(self, x_dataset):
        
        x_dataset=convert2int(x_dataset)
        x_dataset=convert2date(x_dataset)
        x_dataset=new_insights(x_dataset)
        x_dataset=new_insights2(x_dataset)
        

        return x_dataset

In [12]:
df_train

,Store,DayOfWeek,Date,Sales,Open,Promo,StateHoliday,SchoolHoliday
656940,991,3,2013-11-20,5430,1,1,0,0
264573,178,1,2014-12-01,17449,1,1,0,0
406764,261,3,2014-07-02,15924,1,1,0,0
370352,325,7,2014-08-10,0,0,0,0,0
162618,944,7,2015-03-08,0,0,0,0,0
...,...,...,...,...,...,...,...,...
273687,1006,6,2014-11-22,6847,1,0,0,0
884171,762,2,2013-04-30,4970,1,1,0,0
472921,947,7,2014-05-04,0,0,0,0,0
101262,913,6,2015-05-02,10847,1,0,0,0


In [13]:
# pre-processsing step
# Drop the columns - 
# Impute the missing values in column Item_Weight by mean
# Scale the data in the column Item_MRP
pre_process = ColumnTransformer(remainder='passthrough',
                                transformers=[('drop_columns', 'drop', ['Date',
                                                                       #'Customers'
                                                                       ]),
                                                                        
                                                                      
                                              ('scale_data', StandardScaler(),['Difference_to_StateHoliday']),
                                              
                                               ('encoder', OneHotEncoder(), ['Promo','StateHoliday','SchoolHoliday','day','month','weekday','Store','year']) ])
                                             

In [ ]:
# Define the Pipeline
"""
Step1: get the oultet binary columns
Step2: pre processing
Step3: Train a Random Forest Model
"""

rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
model_pipeline = Pipeline(steps=[('get_outlet_binary_columns', OutletTypeEncoder()), 
                                 ('pre_processing',pre_process),
                                 ('random_forest',rf_random)
                                 ])

# fit the pipeline with the training data
model_pipeline.fit(train_x,train_y)

# predict target values on the training data
random_forst=model_pipeline.predict(train_x)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [ ]:
rf_random.best_params_

In [ ]:
predict_train=model_pipeline.predict(train_x)

In [ ]:

print('RMSE on train data: ', mean_squared_error(train_y, random_forst)**(0.5))

In [ ]:
predict_train[:20]

In [ ]:
train_y[:20]

In [ ]:
random_forst[:20]

In [ ]:
df_test = df_test.drop('Id', 1)
df_test

In [ ]:
#df_test=df_test.sample(frac =.0005)


In [ ]:
df_test.isnull().sum()

In [ ]:
df_test['Open'].fillna(df_test['Open'].mode()[0], inplace=True)

In [ ]:
random_forst2=model_pipeline.predict(df_test)